In [3]:
import os
import pandas as pd

if 'dataset.csv' not in os.listdir():
    %run parse_dataset.py
else:
    dataset = pd.read_csv('dataset.csv', dtype=str)
dataset.head()

,IsSummer,HasFlu,HasFoodPoisoning,HasHayFever,HasPneumonia,HasRespiratoryProblems,HasGastricProblems,HasRash,Coughs,IsFatigued,Vomits,HasFever
0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,0,0


In [4]:
from pgmpy.models import BayesianModel
model = BayesianModel()
model.add_edges_from([
 ('HasHayFever', 'HasFever'),
 ('HasHayFever', 'IsFatigued'),
 ('HasHayFever', 'HasRespiratoryProblems'),
 ('HasHayFever', 'HasRash'),
 ('HasHayFever', 'Coughs'),
 ('HasFoodPoisoning', 'HasGastricProblems'),
 ('HasFoodPoisoning', 'Vomits'),
 ('IsSummer', 'HasHayFever'),
 ('HasRespiratoryProblems', 'Coughs'),
 ('HasFlu', 'HasPneumonia'),
 ('HasPneumonia', 'IsFatigued'),
 ('HasPneumonia', 'HasRespiratoryProblems'),
 ('HasPneumonia', 'HasFever'),
 ('HasPneumonia', 'Vomits'),
 ('HasPneumonia', 'HasFoodPoisoning'),
 ('HasPneumonia', 'Coughs')
])

In [5]:
import numpy as np
intrain = np.random.rand(len(dataset)) < 0.999 # About 4000 testing points
model.fit(dataset[intrain])
model.get_cpds():

╒════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╤═══════════════════════════╕
│ HasHayFever            │ HasHayFever(0)            │ HasHayFever(0)            │ HasHayFever(0)            │ HasHayFever(0)            │ HasHayFever(1)            │ HasHayFever(1)            │ HasHayFever(1)            │ HasHayFever(1)            │
├────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┼───────────────────────────┤
│ HasPneumonia           │ HasPneumonia(0)           │ HasPneumonia(0)           │ HasPneumonia(1)           │ HasPneumonia(1)           │ HasPneumonia(0)           │ HasPneumonia(0)           │ HasPneumonia(1)           │ HasPneumonia(1)         

In [14]:
columns = ['IsSummer','HasFlu','HasFoodPoisoning','HasHayFever','HasPneumonia',
        'HasRespiratoryProblems', 'HasGastricProblems', 'HasRash', 'Coughs',
        'IsFatigued', 'Vomits','HasFever']
output = model.predict(dataset[~intrain][[*columns[:4],*columns[5:]]])

In [15]:
len(output)

4092

In [19]:
sum(output.values.ravel() == dataset[~intrain][['HasPneumonia']].values.ravel())

4038

In [21]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(dataset[intrain][[*columns[:4],*columns[5:]]], dataset[intrain][['HasPneumonia']].values.ravel())
output_rfc = clf.predict(dataset[~intrain][[*columns[:4],*columns[5:]]])

In [22]:
sum(output_rfc.ravel() == dataset[~intrain][['HasPneumonia']].values.ravel())

4067